In [1]:
import os

import pandas as pd
import numpy as np
import plotly.express as px

from scipy.stats import pearsonr

In [2]:
URI = os.getenv("DB_URI")

In [3]:
master = pd.read_sql("SELECT * FROM vea_industrial_load_profiles.master", URI)
master.set_index("id", inplace=True)
master.sort_index(inplace=True)

# in the authors database, the following columns are already present in master
# this will lead to problems later on, so we drop them here
# if you have created your own features by the "create_master_features.py", you
# do not need to worry about it
cols_to_drop = ["energy_costs_eur", "capacity_costs_eur", "total_costs_eur"]
for col_to_drop in cols_to_drop:
    try:
        master.drop(columns=col_to_drop, inplace=True)
    except KeyError as e:
        continue

master.head()

,sector_group_id,sector_group,capacity_price_over_2500h_eur_per_kw,energy_price_over_2500h_eur_per_kwh,capacity_price_under_2500h_eur_per_kw,energy_price_under_2500h_eur_per_kwh,zip_code,grid_level,number_of_peak_load,peak_load_kw,mean_load_kw,variance_kw,total_energy_kwh,full_load_hours_h,is_over_2500h,std_kw
id,,,,,,,,,,,,,,,,
0,B,Mining and quarrying,94.60,0.0060,14.67,0.0379,85,MS,1,2227.36,646.151302,423413.418622,5675793.04,2548.215394,True,650.702250
1,B,Mining and quarrying,96.11,0.0052,9.92,0.0397,91,MS,1,1096.00,208.694900,109792.561757,1833176.00,1672.605839,False,331.349607
2,B,Mining and quarrying,96.11,0.0052,9.92,0.0397,92,MS,1,816.00,129.444604,36662.220510,1137041.40,1393.433088,False,191.473812
3,B,Mining and quarrying,96.11,0.0052,9.92,0.0397,85,MS,1,701.60,114.201389,19954.089684,1003145.00,1429.796180,False,141.258946
4,C,Manufacturing industry,101.04,0.0078,12.60,0.0432,30,MS,2,3480.00,1845.483607,610080.465591,16210728.00,4658.255172,True,781.076479


In [4]:
baseline = pd.read_sql("SELECT * FROM vea_results.overview WHERE name LIKE '%%base%%'", URI)
baseline["id"] = baseline["name"].str.split("_").str[0].astype(int)
baseline.set_index("id", inplace=True)
baseline.sort_index(inplace=True)
baseline.head()

,name,energy_costs_eur,grid_energy_costs_eur,grid_capacity_costs_eur,grid_capacity_kw,storage_invest_eur,storage_annuity_eur,storage_capacity_kwh,inverter_invest_eur,inverter_annuity_eur,inverter_capacity_kw,solar_invest_eur,solar_annuity_eur,solar_capacity_kwp,total_yearly_costs_eur,total_annuity_eur,total_invest_eur
id,,,,,,,,,,,,,,,,,
0,0_baseline,1.021643e+06,271558.090967,41249.388603,2227.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.334450e+06,0.0,0.0
1,1_baseline,3.299717e+05,12033.847188,132976.873344,1096.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.749824e+05,0.0,0.0
2,2_baseline,2.046675e+05,7464.085529,99004.679424,816.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.111362e+05,0.0,0.0
3,3_baseline,1.805661e+05,6585.125290,85124.611622,701.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.722758e+05,0.0,0.0
4,4_baseline,2.917931e+06,884063.074775,55353.715200,3480.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.857348e+06,0.0,0.0


In [5]:
storage = pd.read_sql("SELECT * FROM vea_results.overview WHERE name LIKE '%%storage_only'", URI)
storage["id"] = storage["name"].str.split("_").str[0].astype(int)
storage.set_index("id", inplace=True)
storage.sort_index(inplace=True)
storage.head()

,name,energy_costs_eur,grid_energy_costs_eur,grid_capacity_costs_eur,grid_capacity_kw,storage_invest_eur,storage_annuity_eur,storage_capacity_kwh,inverter_invest_eur,inverter_annuity_eur,inverter_capacity_kw,solar_invest_eur,solar_annuity_eur,solar_capacity_kwp,total_yearly_costs_eur,total_annuity_eur,total_invest_eur
id,,,,,,,,,,,,,,,,,
0,0_storage_only,1.018851e+06,270816.128970,41249.388603,2227.360000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.330917e+06,0.000000,0.000000
1,1_storage_only,3.293101e+05,12009.720415,112571.591043,927.818956,72705.633117,5658.350233,255.107485,30272.587940,2355.978453,168.181044,0.0,0.0,0.0,4.619058e+05,8014.328685,102978.221057
2,2_storage_only,2.043740e+05,7453.384611,77439.892170,638.262276,76183.293100,5929.000763,267.309800,31992.790344,2489.854017,177.737724,0.0,0.0,0.0,2.976862e+05,8418.854780,108176.083444
3,3_storage_only,1.800912e+05,6567.806727,75835.912954,625.042224,27228.142451,2119.043045,95.537342,13780.399758,1072.466119,76.557776,0.0,0.0,0.0,2.656864e+05,3191.509164,41008.542208
4,4_storage_only,2.909959e+06,881647.601893,55353.715200,3480.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,3.846960e+06,0.000000,0.000000


## General analysis

In [6]:
total_profiles_analyzed = len(storage)
profiles_using_storage = storage[storage["inverter_invest_eur"] > 0]
n_profiles_using_storage = len(profiles_using_storage)

print(f"{total_profiles_analyzed=}")
print(f"{n_profiles_using_storage=}")
print(f"Percentage of profiles using storage: {((n_profiles_using_storage / total_profiles_analyzed) * 100):.2f} %")

total_profiles_analyzed=5353
n_profiles_using_storage=2853
Percentage of profiles using storage: 53.30 %


## System sizes

In [7]:
stor_cap_col = "storage_capacity_kwh"
median_storage_size = profiles_using_storage[stor_cap_col].median()
print(f"{median_storage_size=:.4f} kWh")
mean_storage_size = profiles_using_storage[stor_cap_col].mean()
print(f"{mean_storage_size=:.4f} kWh")
min_storage_size = profiles_using_storage[stor_cap_col].min()
print(f"{min_storage_size=:.4f} kWh")
max_storage_size = profiles_using_storage[stor_cap_col].max()
print(f"{max_storage_size=:.4f} kWh")
q95_storage_size = profiles_using_storage[stor_cap_col].quantile(0.95)
print(f"{q95_storage_size=:.2f} kWh")

print("----------------------------------")
inv_cap_col = "inverter_capacity_kw"
median_inverter_size = profiles_using_storage[inv_cap_col].median()
print(f"{median_inverter_size=:.2f} kW")
mean_inverter_size = profiles_using_storage[inv_cap_col].mean()
print(f"{mean_inverter_size=:.2f} kW")
min_inverter_size = profiles_using_storage[inv_cap_col].min()
print(f"{min_inverter_size=:.2f} kW")
max_inverter_size = profiles_using_storage[inv_cap_col].max()
print(f"{max_inverter_size=:.2f} kW")
q95_inverter_size = profiles_using_storage[inv_cap_col].quantile(0.95)
print(f"{q95_inverter_size=:.2f} kW")


fig_df = profiles_using_storage.copy()
fig_df = fig_df.rename(columns={stor_cap_col: "Storage", inv_cap_col: "Inverter"})
fig = px.box(
    data_frame=fig_df,
    x=["Inverter", "Storage"],
    title="Battery system sizes")
fig.update_layout(xaxis_title="Capacity in kWh (storage) / kW (inverter)", yaxis_title="")
fig.update_xaxes(range=[0, 200])
fig.show()

median_storage_size=19.7937 kWh
mean_storage_size=80.3042 kWh
min_storage_size=0.0016 kWh
max_storage_size=8539.0705 kWh
q95_storage_size=288.80 kWh
----------------------------------
median_inverter_size=27.79 kW
mean_inverter_size=72.35 kW
min_inverter_size=0.01 kW
max_inverter_size=25586.33 kW
q95_inverter_size=226.92 kW


## System costs

In [8]:
stor_inv_col = "storage_invest_eur"
median_storage_invest = profiles_using_storage[stor_inv_col].median()
print(f"{median_storage_invest=:.2f} €")
mean_storage_invest = profiles_using_storage[stor_inv_col].mean()
print(f"{mean_storage_invest=:.2f} €")
min_storage_invest = profiles_using_storage[stor_inv_col].min()
print(f"{min_storage_invest=:.2f} €")
max_storage_invest = profiles_using_storage[stor_inv_col].max()
print(f"{max_storage_invest=:.2f} €")
q95_storage_invest = profiles_using_storage[stor_inv_col].quantile(0.95)
print(f"{q95_storage_invest=:.2f} €")

print("----------------------------------")
inv_invest_col = "inverter_invest_eur"
median_inverter_invest = profiles_using_storage[inv_invest_col].median()
print(f"{median_inverter_invest=:.2f} €")
mean_inverter_invest = profiles_using_storage[inv_invest_col].mean()
print(f"{mean_inverter_invest=:.2f} €")
min_inverter_invest = profiles_using_storage[inv_invest_col].min()
print(f"{min_inverter_invest=:.2f} €")
max_inverter_invest = profiles_using_storage[inv_invest_col].max()
print(f"{max_inverter_invest=:.2f} €")
q95_inverter_invest = profiles_using_storage[inv_invest_col].quantile(0.95)
print(f"{q95_inverter_invest=:.2f} €")

fig_df = profiles_using_storage.copy()
fig_df = fig_df.rename(columns={stor_inv_col: "Storage", inv_invest_col: "Inverter"})
fig = px.box(
    data_frame=fig_df,
    x=["Inverter", "Storage"],
    title="Battery system investments")
fig.update_layout(xaxis_title="Storage system investments in €", yaxis_title="")
fig.update_xaxes(range=[0, 50000])
fig.show()

median_storage_invest=5641.20 €
mean_storage_invest=22886.69 €
min_storage_invest=0.45 €
max_storage_invest=2433635.08 €
q95_storage_invest=82307.87 €
----------------------------------
median_inverter_invest=5001.82 €
mean_inverter_invest=13022.16 €
min_inverter_invest=1.08 €
max_inverter_invest=4605540.10 €
q95_inverter_invest=40845.60 €


## Savings

In [9]:
abs_diff = baseline.drop(columns="name") - storage.drop(columns="name")
abs_diff.head()

,energy_costs_eur,grid_energy_costs_eur,grid_capacity_costs_eur,grid_capacity_kw,storage_invest_eur,storage_annuity_eur,storage_capacity_kwh,inverter_invest_eur,inverter_annuity_eur,inverter_capacity_kw,solar_invest_eur,solar_annuity_eur,solar_capacity_kwp,total_yearly_costs_eur,total_annuity_eur,total_invest_eur
id,,,,,,,,,,,,,,,,
0,2791.373626,741.961997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,3533.335623,0.000000,0.000000
1,661.563319,24.126773,20405.282301,168.181044,-72705.633117,-5658.350233,-255.107485,-30272.587940,-2355.978453,-168.181044,0.0,0.0,0.0,13076.643708,-8014.328685,-102978.221057
2,293.422377,10.700918,21564.787254,177.737724,-76183.293100,-5929.000763,-267.309800,-31992.790344,-2489.854017,-177.737724,0.0,0.0,0.0,13450.055769,-8418.854780,-108176.083444
3,474.880156,17.318563,9288.698668,76.557776,-27228.142451,-2119.043045,-95.537342,-13780.399758,-1072.466119,-76.557776,0.0,0.0,0.0,6589.388223,-3191.509164,-41008.542208
4,7972.489180,2415.472882,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,10387.962062,0.000000,0.000000


In [10]:
rel_diff = (baseline.drop(columns="name") - storage.drop(columns="name")) / baseline.drop(columns="name")

# drop those that could not be optimized
rel_diff.dropna(subset="total_yearly_costs_eur", inplace=True)

rel_diff

,energy_costs_eur,grid_energy_costs_eur,grid_capacity_costs_eur,grid_capacity_kw,storage_invest_eur,storage_annuity_eur,storage_capacity_kwh,inverter_invest_eur,inverter_annuity_eur,inverter_capacity_kw,solar_invest_eur,solar_annuity_eur,solar_capacity_kwp,total_yearly_costs_eur,total_annuity_eur,total_invest_eur
id,,,,,,,,,,,,,,,,
0,0.002732,0.002732,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002648,NaN,NaN
1,0.002005,0.002005,0.153450,0.153450,-inf,-inf,-inf,-inf,-inf,-inf,NaN,NaN,NaN,0.027531,-inf,-inf
2,0.001434,0.001434,0.217816,0.217816,-inf,-inf,-inf,-inf,-inf,-inf,NaN,NaN,NaN,0.043229,-inf,-inf
3,0.002630,0.002630,0.109119,0.109119,-inf,-inf,-inf,-inf,-inf,-inf,NaN,NaN,NaN,0.024201,-inf,-inf
4,0.002732,0.002732,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002693,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5354,0.002732,0.002732,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002704,NaN,NaN
5355,0.002730,0.002730,0.037830,0.037830,-inf,-inf,-inf,-inf,-inf,-inf,NaN,NaN,NaN,0.003182,-inf,-inf
5356,0.002732,0.002732,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002710,NaN,NaN


In [11]:
tot_y_sav_col = "total_yearly_costs_eur"
median_yearly_savings = abs_diff[tot_y_sav_col].median()
print(f"{median_yearly_savings=:.2f} €")
mean_yearly_savings = abs_diff[tot_y_sav_col].mean()
print(f"{mean_yearly_savings=:.2f} €")
min_yearly_savings = abs_diff[tot_y_sav_col].min()
print(f"{min_yearly_savings=:.2f} €")
max_yearly_savings = abs_diff[tot_y_sav_col].max()
print(f"{max_yearly_savings=:.2f} €")
q95_yearly_savings = abs_diff[tot_y_sav_col].quantile(0.95)
print(f"{q95_yearly_savings=:.2f} €")

fig_df = abs_diff.copy()
fig_df = fig_df.rename(columns={tot_y_sav_col: "Savings"})
fig = px.box(
    data_frame=fig_df,
    x=["Savings"],
    title="Total yearly savings")
fig.update_layout(xaxis_title="Total yearly savings in €", yaxis_title="")
fig.update_xaxes(range=[0, 10000])
fig.show()

median_yearly_savings=1378.72 €
mean_yearly_savings=3613.75 €
min_yearly_savings=0.00 €
max_yearly_savings=514178.43 €
q95_yearly_savings=12927.54 €


In [12]:
perc_yearly_savings = (abs_diff["total_yearly_costs_eur"] / baseline["total_yearly_costs_eur"]) * 100
median_perc_yearly_savings = perc_yearly_savings.median()
print(f"{median_perc_yearly_savings=:.2f} %")
mean_perc_yearly_savings = perc_yearly_savings.mean()
print(f"{mean_perc_yearly_savings=:.2f} %")
min_perc_yearly_savings = perc_yearly_savings.min()
print(f"{min_perc_yearly_savings=:.2f} %")
max_perc_yearly_savings = perc_yearly_savings.max()
print(f"{max_perc_yearly_savings=:.2f} %")
q95_perc_yearly_savings = perc_yearly_savings.quantile(0.8)
print(f"{q95_perc_yearly_savings=:.2f} %")
print("-----------------------")
n_profiles_1pct_yearly_savings = len(perc_yearly_savings[perc_yearly_savings > 1])
print(f"Number of profiles with more than 1% yearly savings: {n_profiles_1pct_yearly_savings}")
perc_profiles_1pct_yearly_savings = (n_profiles_1pct_yearly_savings / len(perc_yearly_savings)) * 100
print(f"Percentage of profiles with more than 1% yearly savings: {perc_profiles_1pct_yearly_savings:.2f} %")
n_profiles_2pct_yearly_savings = len(perc_yearly_savings[perc_yearly_savings > 2])
print(f"Number of profiles with more than 2% yearly savings: {n_profiles_2pct_yearly_savings}")
perc_profiles_2pct_yearly_savings = (n_profiles_2pct_yearly_savings / len(perc_yearly_savings)) * 100
print(f"Percentage of profiles with more than 2% yearly savings: {perc_profiles_2pct_yearly_savings:.2f} %")
n_profiles_3pct_yearly_savings = len(perc_yearly_savings[perc_yearly_savings > 3])
print(f"Number of profiles with more than 3% yearly savings: {n_profiles_3pct_yearly_savings}")
perc_profiles_3pct_yearly_savings = (n_profiles_3pct_yearly_savings / len(perc_yearly_savings)) * 100
print(f"Percentage of profiles with more than 3% yearly savings: {perc_profiles_3pct_yearly_savings:.2f} %")


fig_df = pd.DataFrame()
fig_df["Savings"] = perc_yearly_savings.copy()
fig_df = fig_df.rename(columns={"Savings": "Savings"})
fig = px.box(
    data_frame=fig_df,
    x="Savings",
    title="Relative yearly savings")
fig.update_layout(xaxis_title="Relative yearly savings in %", yaxis_title="")
fig.update_xaxes(range=[0, 6])
fig.show()

median_perc_yearly_savings=0.27 %
mean_perc_yearly_savings=1.53 %
min_perc_yearly_savings=0.24 %
max_perc_yearly_savings=75.06 %
q95_perc_yearly_savings=2.25 %
-----------------------
Number of profiles with more than 1% yearly savings: 1560
Percentage of profiles with more than 1% yearly savings: 29.11 %
Number of profiles with more than 2% yearly savings: 1152
Percentage of profiles with more than 2% yearly savings: 21.50 %
Number of profiles with more than 3% yearly savings: 835
Percentage of profiles with more than 3% yearly savings: 15.58 %


In [ ]:
yearly_cap_cost_sav_col = "grid_capacity_costs_eur"
median_yearly_cap_cost_savings = abs_diff[yearly_cap_cost_sav_col].median()
print(f"{median_yearly_cap_cost_savings=:.2f} €")
mean_yearly_cap_cost_savings = abs_diff[yearly_cap_cost_sav_col].mean()
print(f"{mean_yearly_cap_cost_savings=:.2f} €")
min_yearly_cap_cost_savings = abs_diff[yearly_cap_cost_sav_col].min()
print(f"{min_yearly_cap_cost_savings=:.2f} €")
max_yearly_cap_cost_savings = abs_diff[yearly_cap_cost_sav_col].max()
print(f"{max_yearly_cap_cost_savings=:.2f} €")
q95_yearly_cap_cost_savings = abs_diff[yearly_cap_cost_sav_col].quantile(0.95)
print(f"{q95_yearly_cap_cost_savings=:.2f} €")

fig_df = abs_diff.copy()
fig_df = fig_df.rename(columns={tot_y_sav_col: "Savings"})
fig = px.box(
    data_frame=fig_df,
    x="Savings",
    title="Absolute yearly capacity costs savings")
fig.update_layout(xaxis_title="Savings in €", yaxis_title="")
fig.update_xaxes(range=[0, 10e3])
fig.show()

median_yearly_cap_cost_savings=73.52 €
mean_yearly_cap_cost_savings=3242.98 €
min_yearly_cap_cost_savings=0.00 €
max_yearly_cap_cost_savings=1061384.17 €
q95_yearly_cap_cost_savings=13505.03 €


In [14]:
perc_yearly_cap_cost_savings = (abs_diff["grid_capacity_costs_eur"] / baseline["grid_capacity_costs_eur"]) * 100
median_perc_yearly_cap_cost_savings = perc_yearly_cap_cost_savings.median()
print(f"{median_perc_yearly_cap_cost_savings=:.2f} %")
mean_perc_yearly_cap_cost_savings = perc_yearly_cap_cost_savings.mean()
print(f"{mean_perc_yearly_cap_cost_savings=:.2f} %")
min_perc_yearly_cap_cost_savings = perc_yearly_cap_cost_savings.min()
print(f"{min_perc_yearly_cap_cost_savings=:.2f} %")
max_perc_yearly_cap_cost_savings = perc_yearly_cap_cost_savings.max()
print(f"{max_perc_yearly_cap_cost_savings=:.2f} %")
q95_perc_yearly_cap_cost_savings = perc_yearly_cap_cost_savings.quantile(0.8)
print(f"{q95_perc_yearly_cap_cost_savings=:.2f} %")
print("-----------------------")
n_profiles_1pct_yearly_cap_cost_savings = len(perc_yearly_cap_cost_savings[perc_yearly_cap_cost_savings > 1])
print(f"Number of profiles with more than 1% yearly savings: {n_profiles_1pct_yearly_cap_cost_savings}")
perc_profiles_1pct_yearly_cap_cost_savings = (len(perc_yearly_cap_cost_savings[perc_yearly_cap_cost_savings > 1]) / len(perc_yearly_savings)) * 100
print(f"Percentage of profiles with more than 1% yearly savings: {perc_profiles_1pct_yearly_cap_cost_savings:.2f} %")
n_profiles_2pct_yearly_cap_cost_savings = len(perc_yearly_cap_cost_savings[perc_yearly_cap_cost_savings > 2])
print(f"Number of profiles with more than 2% yearly savings: {n_profiles_2pct_yearly_cap_cost_savings}")
perc_profiles_2pct_yearly_cap_cost_savings = (len(perc_yearly_cap_cost_savings[perc_yearly_cap_cost_savings > 2]) / len(perc_yearly_savings)) * 100
print(f"Percentage of profiles with more than 2% yearly savings: {perc_profiles_2pct_yearly_cap_cost_savings:.2f} %")


fig_df = pd.DataFrame()
fig_df["Savings"] = perc_yearly_savings.copy()
fig_df = fig_df.rename(columns={"Savings": "Savings"})
fig = px.box(
    data_frame=fig_df,
    x="Savings",
    title="Relative yearly capacity costs savings")
fig.update_layout(xaxis_title="Savings in %", yaxis_title="")
fig.update_xaxes(range=[0, 6])
fig.show()

median_perc_yearly_cap_cost_savings=1.16 %
mean_perc_yearly_cap_cost_savings=9.21 %
min_perc_yearly_cap_cost_savings=0.00 %
max_perc_yearly_cap_cost_savings=99.97 %
q95_perc_yearly_cap_cost_savings=19.13 %
-----------------------
Number of profiles with more than 1% yearly savings: 2705
Percentage of profiles with more than 1% yearly savings: 50.48 %
Number of profiles with more than 2% yearly savings: 2515
Percentage of profiles with more than 2% yearly savings: 46.93 %


## Correlations to absolute savings

In [15]:
# merge savings onto master (with features)
abs_diff_with_master = pd.merge(left=abs_diff, right=master, how="left", left_index=True, right_index=True)
abs_diff_with_master.head()

abs_diff_with_master["std_by_mean"] = abs_diff_with_master["std_kw"] / abs_diff_with_master["mean_load_kw"]
abs_diff_with_master["peak_by_mean"] = abs_diff_with_master["peak_load_kw"] / abs_diff_with_master["mean_load_kw"]

In [16]:
cols_to_drop = [
    "grid_level",
    "zip_code",
    "sector_group_id",
    "sector_group",
    "solar_invest_eur",
    "solar_annuity_eur",
    "solar_capacity_kwp"]
abs_correlations_df = abs_diff_with_master.drop(columns=cols_to_drop).corr()
px.imshow(abs_correlations_df, title="Correlation coefficients for total yearly savings")

In [17]:
fig_df = abs_correlations_df[["total_yearly_costs_eur"]].round(2)
fig_df.sort_values("total_yearly_costs_eur", inplace=True, ascending=False)
fig = px.bar(
    data_frame=fig_df,
    y="total_yearly_costs_eur",
    text_auto=True,
    title="Correlation between different load profile characteristics and total yearly savings")
fig.update_layout(yaxis_title="Correlation coefficient", xaxis_title="Variable")

In [18]:
df = pd.DataFrame()
i = 0
for var in abs_correlations_df.index:#[correlations_df["total_yearly_costs_eur"] > 0.3].index:
    corr, p_value = pearsonr(y=abs_diff_with_master.dropna()["total_yearly_costs_eur"], x=abs_diff_with_master.dropna()[var])
    df.loc[i, "var"] = var
    df.loc[i, "corr"] = corr
    df.loc[i, "p_value"] = p_value
    i += 1

df.sort_values("corr", ascending=False, ignore_index=True)

,var,corr,p_value
0,total_yearly_costs_eur,1.000000,0.000000e+00
1,grid_capacity_costs_eur,0.870315,0.000000e+00
2,grid_capacity_kw,0.854124,0.000000e+00
3,peak_load_kw,0.580533,0.000000e+00
4,std_kw,0.489817,4.102527e-310
5,total_energy_kwh,0.446129,2.717942e-251
6,mean_load_kw,0.446129,2.717942e-251
7,energy_costs_eur,0.365821,2.075243e-163
8,grid_energy_costs_eur,0.347917,5.858155e-147
9,variance_kw,0.319883,2.793086e-123


## Correlation to relative savings

In [19]:
# merge savings onto master (with features)
rel_diff_with_master = pd.merge(left=rel_diff, right=master, how="left", left_index=True, right_index=True)
rel_diff_with_master.head()

rel_diff_with_master["std_by_mean"] = rel_diff_with_master["std_kw"] / rel_diff_with_master["mean_load_kw"]
rel_diff_with_master["std_by_peak"] = rel_diff_with_master["std_kw"] / rel_diff_with_master["peak_load_kw"]
rel_diff_with_master["peak_by_mean"] = rel_diff_with_master["peak_load_kw"] / rel_diff_with_master["mean_load_kw"]

In [20]:
cols_to_drop = [
    "grid_level",
    "zip_code",
    "sector_group_id",
    "sector_group",
    "solar_invest_eur",
    "solar_annuity_eur",
    "solar_capacity_kwp"]
rel_correlations_df = rel_diff_with_master.drop(columns=cols_to_drop).corr()
px.imshow(rel_correlations_df, title="Correlation coefficients for relative yearly savings")

In [21]:
fig_df = rel_correlations_df[["total_yearly_costs_eur"]].round(2)
fig_df.sort_values("total_yearly_costs_eur", inplace=True, ascending=False)
fig_df.dropna(inplace=True)
fig = px.bar(
    data_frame=fig_df,
    y="total_yearly_costs_eur",
    text_auto=True,
    title="Correlation between different load profile characteristics and relative yearly savings")
fig.update_layout(yaxis_title="Correlation coefficient", xaxis_title="Variable")

In [22]:
df = pd.DataFrame()
i = 0
for var in rel_correlations_df.index:
    if np.inf in rel_diff_with_master[var]:
        continue
    elif -np.inf in rel_diff_with_master[var]:
        continue
    elif np.nan in rel_diff_with_master[var]:
        continue
    elif rel_diff_with_master[var].isin([np.nan, np.inf, -np.inf]).any():
        continue
    if "storage" in var or "inverter" in var:
        continue

    corr, p_value = pearsonr(y=rel_diff_with_master["total_yearly_costs_eur"], x=rel_diff_with_master[var])
    df.loc[i, "var"] = var
    df.loc[i, "corr"] = corr
    df.loc[i, "p_value"] = p_value
    i += 1

df.sort_values("corr", ascending=False, ignore_index=True)

,var,corr,p_value
0,total_yearly_costs_eur,1.000000,0.000000e+00
1,grid_capacity_costs_eur,0.813182,0.000000e+00
2,grid_capacity_kw,0.813182,0.000000e+00
3,std_by_mean,0.487954,2.678775e-318
4,peak_by_mean,0.377786,3.632898e-181
5,energy_price_under_2500h_eur_per_kwh,0.214763,7.014716e-57
6,capacity_price_over_2500h_eur_per_kw,0.192926,4.836621e-46
7,capacity_price_under_2500h_eur_per_kw,0.083816,8.192652e-10
8,energy_price_over_2500h_eur_per_kwh,0.072840,9.575004e-08
9,number_of_peak_load,-0.021791,1.109388e-01
